In [1]:
from diseal import seal
import numpy as np

In [2]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)

poly_mod = 8192일 때, coeff_Modulus가 총합 얼마까지 가능? 

In [ ]:
op = {"scheme":scheme, "relin":relin,"batch":batch,"level":level,"equation":equation,"rotate":rotate} 

if op["batch"] is True: 

    print(f"Q = {Q}\nTotal Bit Length = {sum(Q)}") 

    print(f"Max Bit Count = {seal.CoeffModulus.MaxBitCount(N)}") 

    if sum(Q) > seal.CoeffModulus.MaxBitCount(N) :  

        print(f"Bit length  should be limited to MaxBitCount.") 

 

if op["scheme"]==ckks: 

    for i in op: 

        if op[i]==False: 

            print("[WARNING]CKKS comes with relin/batch/level.") 

 

 

 

 

 

 

 

#--------------------------- Get Variable Name String 

def namestr(obj,namespace): 

    get_name=[name for name in namespace if namespace[name] is obj] 

    return get_name[0] 

 

#--------------------------- [seal] Get parms_id   

def get_parms_id(var): 

    return context.get_context_data(var.parms_id()) 
 

#--------------------------- [seal] Get bit lenght of the scale  

def nscales(operand): 

    return np.round(np.log2(operand.scale())) 


#--------------------------- [seal] Get chain index 

def chain_idx(var): 

    return get_parms_id(var).chain_index() 

 
#--------------------------- [seal] info : size(dimension), chain index, scale, noise_budget(bfv/bgv only)  

def check_point(var): 

    print(f"\t[ CHECK ] ")#"{namestr(var,globals())} ----------------") 

    print(f'\t[  Size   ] {var.size()} [bit]') 

    print(f'\t[ Chain ID] {chain_idx(var)}')   

    print(f'\t[ Scale   ] {nscales(var)}') 

    if(op["scheme"]==bfv or op["scheme"]==bgv): 

        budget = decryptor.invariant_noise_budget(var) 

        print(f'\t[Encrypted] budget = {budget} [bit]\n')     


#--------------------------- print 1D vector into 2D matrix 

def print_matrix(mat,nrows,nprecision): 

    np.set_printoptions(precision=nprecision) 

    M = list() 

    dim = np.size(mat) 

    chunk_size = int(np.round(1.0*dim/nrows)) 

    for i in range(0,dim,chunk_size): 

        M.append(mat[i:i+chunk_size])   

    print(M)   

 
#--------------------------- [seal] copy variables when manipulating them only within functions 

def copy_input(x_org): 

    if type(x_org)==seal.Ciphertext:         

        x = seal.Ciphertext(x_org) 

    elif type(x_org)==seal.Plaintext:        

        x = seal.Plaintext(x_org) 

    else:  

        x = x_org 

    return x 

 
#--------------------------- [seal] match scales of elements in xs = [c,...] 

def match_scales(xs,names): 

    while(nscales(xs[0]) != nscales(xs[1])): 

        idx = int(~(nscales(xs[0]) > nscales(xs[1]))) 

        evaluator.rescale_to_next_inplace(xs[idx]) 

        #print(f"\t\t\t\t[rescaled] {names[idx]} to {nscales(xs[idx])}") 

    return xs 



#--------------------------- [seal] match levels (chain index) of elements in xs = [c,...] 

def match_levels(xs,names): 

    #print(f"[0](lv1,lv2)=({chain_idx(xs[0])},{chain_idx(xs[1])})") 

    while(chain_idx(xs[0])!=chain_idx(xs[1])): 

        idx = int(~( chain_idx(xs[0]) > chain_idx(xs[1]))) 

        evaluator.mod_switch_to_inplace(xs[idx],xs[~idx].parms_id()) 

        #print(f"\t\t\t\t[ mod sw ]{names[idx]} to {chain_idx(xs[idx])}") 

    return xs 

In [60]:
def check_ctxt(ctxt):
    cd = context.get_context_data(ctxt.parms_id())
    print("parms id", cd.parms_id())
    print("Scale", np.log2(ctxt.scale()))

In [57]:
def decrypt(ctxt):
    decp = decryptor.decrypt(ctxt)
    return encoder.decode(decp)

def _sum_reduce_seal(evaluator, galois_keys, nslots, ctx: seal.Ciphertext, n=None):
    """return the sum of a Ciphertext (repeated nslot times), SEAL version
    """        
    if n is None:
        n = int(np.log2(nslots))

    temp = seal.Ciphertext()
    output = seal.Ciphertext()

    for i in range(n):
        if i == 0:
            temp = evaluator.rotate_vector(ctx, 2**i, galois_keys)
            output = evaluator.add(ctx, temp)
        else:
            temp = evaluator.rotate_vector(output, 2**i, galois_keys)
            evaluator.add_inplace(output, temp)
    return output

In [64]:
def eval_mean(ctxt):
    """
    # Is it faster to give only one value than nslot-long plain text?
    """
    ctxt = _sum_reduce_seal(evaluator, gk, nslots, ctxt)
    evaluator.multiply_plain_inplace(ctxt, encoder.encode([1/nslots], 2**scale))
    return ctxt

In [8]:
poly_modulus_degree = 8192*2
scale = 30

coeff_scales = [60] + [scale]*8 +[60]
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, coeff_scales))

In [10]:
context = seal.SEALContext(parms)
keygenerator = seal.KeyGenerator(context)
sk = keygenerator.secret_key()

In [11]:
pk = keygenerator.create_public_key()
gk = keygenerator.create_galois_keys()
rk = keygenerator.create_relin_keys()

In [12]:
encryptor = seal.Encryptor(context, pk)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, sk)
encoder   = seal.CKKSEncoder(context)

In [13]:
nslots = encoder.slot_count()

In [75]:
arr = np.random.rand(nslots) + 3
print(arr)
ptxt = encoder.encode(arr, 2**scale)

[3.50853717 3.69029299 3.62067451 ... 3.64165775 3.32951568 3.75489862]


In [100]:
ctxt = encryptor.encrypt(ptxt)

In [101]:
mean = eval_mean(ctxt)

In [102]:
evaluator.rescale_to_next_inplace(mean)

In [103]:
check_ctxt(mean)
decrypt(mean)

parms id [7124486971406016613, 9200588442819745994, 9633530931110124529, 17700107536045618697]
Scale 30.00013208737876


array([ 3.50757826e+00,  5.73187725e-03,  2.69910104e-03, ...,
       -7.90402264e-05, -3.76156212e-05, -2.69679731e-05])

In [104]:
dec = decrypt(ctxt)

In [105]:
print(dec[0])

3.5085379886027135


In [106]:
check_ctxt(ctxt)

parms id [13137530424728705773, 5253044371671211702, 18366098092035193399, 10828290947916765783]
Scale 30.0


In [107]:
scaled = evaluator.mod_switch_to_next(ctxt)

In [110]:
check_ctxt(scaled)
check_ctxt(mean)

parms id [7124486971406016613, 9200588442819745994, 9633530931110124529, 17700107536045618697]
Scale 30.0
parms id [7124486971406016613, 9200588442819745994, 9633530931110124529, 17700107536045618697]
Scale 30.00013208737876


In [109]:
subtracted = evaluator.sub(scaled, mean)

ValueError: scale mismatch